In [32]:
day1_filepath = '../data/day_01_input.csv'
day2_filepath = '../data/day_02_input.csv'
day3_filepath = '../data/day_03_input.csv'
# day1_filepath = '../data/day_01_input.csv'
# day1_filepath = '../data/day_01_input.csv'
# day1_filepath = '../data/day_01_input.csv'

In [12]:
# day 1
with open(day1_filepath) as file:
    num_increasing = 0
    # num_decreasing = 0
    step = 1
    previous_depth = int(file.readline())
    while True:
        try:
            current_depth = int(file.readline())
            step += 1
        except: 
            break
        diff = previous_depth - current_depth
        if diff < 0:
            num_increasing +=1
        # else:
        #     num_decreasing +=1
        previous_depth = current_depth

print('read in', step, 'depth measurements and found', num_increasing, 'of them increasing over previous step')

read in 2000 depth measurements and found 1602 of them increasing over previous step


Considering every single measurement isn't as useful as you expected: there's just too much noise in the data. Instead, consider sums of a three-measurement sliding window. Again considering the above example:

```
199  A
200  A B
208  A B C
210    B C D
200  E   C D
207  E F   D
240  E F G
269    F G H
260      G H
263        H
```
Start by comparing the first and second three-measurement windows. The measurements in the first window are marked `A (199, 200, 208)`; their sum is `199 + 200 + 208 = 607`. The second window is marked `B (200, 208, 210)`; its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare `A` with `B,` then compare `B` with `C`, then `C` with `D`, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

In the above example, the sum of each three-measurement window is as follows:
```
A: 607 (N/A - no previous sum)
B: 618 (increased)
C: 618 (no change)
D: 617 (decreased)
E: 647 (increased)
F: 716 (increased)
G: 769 (increased)
H: 792 (increased)
```
In this example, there are 5 sums that are larger than the previous sum.
Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [13]:
with open(day1_filepath) as file:
    num_increasing = 0
    # num_decreasing = 0
    previous_window = []
    for _ in range(3):
        previous_window.append(int(file.readline()))
    current_window = previous_window[-2:]
    next_window = [previous_window[-1]]
    num_steps = 3
    # while num_steps <10:
    while True:
        try:
            current_window.append(int(file.readline()))
        except:
            break
        num_steps +=1
        next_window.append(current_window[-1])
        # print(
        #     'A', previous_window, sum(previous_window),
        #     'B', current_window,  sum(current_window),
        #     'C', next_window, sum(next_window)
        # )
        diff = sum(previous_window) - sum(current_window)
        if diff < 0:
            num_increasing +=1
        previous_window = current_window
        current_window  = next_window
        next_window     = [current_window[-1]]

print('read in', num_steps, 'depth measurements and found', num_increasing, ' 3-measurement windows increasing over the previous')

read in 2000 depth measurements and found 1633  3-measurement windows increasing over the previous


In [14]:
# day 2 second version
window_size = 3
num_increasing = 0
running_window = []
with open(day1_filepath) as file:
    # initialise running window of 4 numbers
    for _ in range(window_size):
        running_window.append(int(file.readline()))
    step_num = window_size
    while True:
        try:
            running_window.append(int(file.readline()))
        except:
            break
        step_num +=1
        diff = sum(running_window[:window_size]) - sum(running_window[-window_size:])
        if diff < 0: # current>previous
            num_increasing +=1
        del running_window[0]

print('read in', step_num, 'depth measurements and found', 
    num_increasing, 'measurement windows of size', window_size, 
    'increasing depth over the previous window'
)

read in 2000 depth measurements and found 1633 measurement windows of size 3  increasing over the previous window


# --- Day 2: Dive! ---

Now, you need to figure out how to pilot this thing. It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

- `forward X` increases the horizontal position by X units.
- `down X` increases the depth by X units.
- `up X` decreases the depth by X units.

Note that since you're on a submarine, `down` and `up` affect your depth, and so they have the opposite result of what you might expect. The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```
Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

- `forward 5` adds 5 to your horizontal position, a total of 5.
- `down 5` adds 5 to your depth, resulting in a value of 5.
- `forward 8` adds 8 to your horizontal position, a total of 13.
- `up 3` decreases your depth by 3, resulting in a value of 2.
- `down 8` adds 8 to your depth, resulting in a value of 10.
- `forward 2` adds 2 to your horizontal position, a total of 15.
After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [17]:
x,d,step = 0,0,0
with open(day2_filepath) as file:
# with open('data/temp') as file:
    while True:
        try:
            direction, stepsize = file.readline().split(' ')
        except: 
            break
        step +=1
        stepsize = int(stepsize)
        if direction == 'forward':
            x += stepsize
        elif direction == 'down':
            d += stepsize
        elif direction == 'up':
            d -= stepsize

print('step ', step, ':', direction, 'by', stepsize, ':-----> (', x, ',', d,')')
print('multiplied:', x*d)

step  1000 : forward by 6 :-----> ( 2007 , 747 )
multiplied: 1499229


# --- Part Two ---

Based on your calculations, the planned course does not to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal `position` and `depth`, you'll also need to track a third value, `aim`, which also starts at 0. The commands also mean something entirely different than you first thought:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
    + It increases your horizontal position by X units.
    + It increases your depth by your aim multiplied by X.
Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.
Now, the above example does something different:

- `forward 5` adds 5 to your horizontal position, a total of 5. Because your `aim` is 0, your `depth` does not change.
- `down 5` adds 5 to your `aim`, resulting in a value of 5.
- `forward 8` adds 8 to your horizontal position, a total of 13. Because your `aim` is 5, your `depth` increases by 8x5=40.
- `up 3` decreases your aim by 3, resulting in a value of 2.
- `down 8` adds 8 to your aim, resulting in a value of 10.
- `forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2x10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [21]:
x,aim,depth,step = 0,0,0,0
with open(day2_filepath) as file:
# with open('data/temp') as file:
    while True:
        try:
            direction, stepsize = file.readline().split(' ')
        except: 
            break
        step +=1
        stepsize = int(stepsize)
        if direction == 'forward':
            x += stepsize
            depth += stepsize*aim
        elif direction == 'down':
            aim += stepsize
        elif direction == 'up':
            aim -= stepsize

print('step ', step, ':', direction, 'by', stepsize, ':-----> (', x, ',', depth,')')
print('multiplied:', x*depth)

step  1000 : forward by 6 :-----> ( 2007 , 668080 )
multiplied: 1340836560


# --- Day 3: Binary Diagnostic ---

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.
The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.
You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.
Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:
```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```
Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.
The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.
The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.
So, the gamma rate is the binary number `10110`, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [44]:
from operator import add
with open(day3_filepath) as file:
    line = 1
    first_line = file.readline().rstrip('\n')
    counts = [int(bit) for bit in first_line]
    # print(line, first_line, counts)
    # while line <19:
    while True:
        new_line = file.readline().rstrip('\n')
        if not new_line:
            break
        line +=1
        counts = list(map(add, counts, [int(bit) for bit in new_line]))
        # print(line, new_line, counts)
        
half = line/2
print(line, half, counts)
test_list = [1 if count == half else 0 for count in counts]
if sum(test_list) > 0:    
    print('warning! a draw is unexpected!', line, half, test_list, counts)
gamma_list = [1 if count > half  else 0 for count in counts]
gamma = int("".join(str(bit) for bit in gamma_list),2)
epsilon_list = [1 if count < half  else 0 for count in counts]
epsilon = int("".join(str(bit) for bit in epsilon_list),2)
nrg = gamma * epsilon

print('gamma_list', gamma_list)
print('epsilon_list', epsilon_list)
print('test_list', test_list)
print('gamma', gamma)
print('epsilon', epsilon)
print('energy', n)

1000 [497, 509, 520, 507, 491, 496, 511, 492, 505, 516, 478, 497]
gamma_list [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
epsilon_list [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1]
test_list [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
gamma 1836
epsilon 2259
energy 4147524


In [20]:
vec = [1,0,1,0,1,0]
flip = [1 if bit == 0 else 0 for bit in vec]
n= int("".join(str(bit) for bit in vec),2) * int("".join(str(bit) for bit in flip),2)
print(n)
b= []
while(n>0):
    d=n%2
    b.append(d)
    n=n//2
b.reverse()
print("multiply", vec, 'and', flip, 'to get')
for i in b:
    print(i,end="")
    
def fn(s, base=10):
    prefix = s[0:2]
    if prefix == '0x':
        base = 16
    elif prefix == '0b':
        base = 2
    return bin(int(s, base))


print(fn('15'))
print(fn('0xF'))
print(fn('0b1111'))


882
multiply [1, 0, 1, 0, 1, 0] and [0, 1, 0, 1, 0, 1] to get
1101110010

# --- Part Two ---
Next, you should verify the life support rating, which can be determined by multiplying the _oxygen generator rating_ by the _CO2 scrubber_ rating.

Both the _oxygen generator rating_ and the _CO2 scrubber rating_ are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

- Keep only numbers selected by the _bit criteria_ for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.

The _bit criteria_ depends on which type of rating value you want to find:
- To find _oxygen generator rating_, determine the most common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `1` in the position being considered.
- To find _CO2 scrubber rating_, determine the least common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `0` in the position being considered.

For example, to determine the _oxygen generator rating_ value using the same example diagnostic report from above:
- Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
- Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
- In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
- In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
- In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the oxygen generator rating, keep the number with a `1` in that position: `10111`.
- As there is only one number left, stop; the _oxygen generator rating_ is `10111`, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a 0 in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
- Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
- In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the CO2 scrubber rating, keep the number with a `0` in that position: `01010`.
- As there is only one number left, stop; the _CO2 scrubber rating_ is `01010`, or 10 in decimal.
- Finally, to find the life support rating, multiply the _oxygen generator rating_ (23) by the _CO2 scrubber rating_ (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [15]:
with open('../data/temp') as file:
    # read in once and store in memory
    candidates = [bit_str.rstrip('\n') for bit_str in file.readlines()]
# could use counts from previous calculation, or just recompute it
len_num = len(candidates[0]) # assume uniform data. length of first is length of all

def get_rating(candidates, system='oxygen'):
    rating = None
    for digit in range(len_num):
        num_num = len(candidates) # number of binary numbers to search
        if num_num == 1:
            rating = candidates[0]
            break
        num_ones = 0
        for number in range(num_num):
            num_ones += int(candidates[number][digit])
        if num_ones >= num_num/2:
            # 1 is the majority digit, or they are even
            oxygen_filter = '1'
            co2_filter = '0'
        else:
            # 0 is the majority digit
            oxygen_filter = '0'
            co2_filter = '1'
        if system == 'oxygen':
            candidates = [number for number in candidates if number[digit] == oxygen_filter]
        elif system == 'co2':
            candidates = [number for number in candidates if number[digit] == co2_filter]
        else:
            print('unsupported system! only oxygen or co2 are supported, you entered:', system)
    if num_num != 1:
        print('how did this happen?', candidates)
    return rating

print(get_rating(candidates, 'oxygen'))

101001000111


In [13]:
with open('../data/temp') as file:
    # read in once and store in memory
    candidates = [bit_str.rstrip('\n') for bit_str in file.readlines()]
oxygen_filter = '1'
[number for number in candidates if number[0] == oxygen_filter]

['100111000110',
 '101101000100',
 '110111100010',
 '111000010011',
 '111110101101',
 '110100101011',
 '100100011010',
 '110111110100',
 '100011011110',
 '111111111010',
 '110100000110',
 '110011010011',
 '101011001011',
 '110111100001',
 '100001001001',
 '111100000011',
 '110110110111',
 '101000011011',
 '110100100000',
 '110101101001',
 '111101001111',
 '110001000001',
 '101100011110',
 '111001010011',
 '101000010100',
 '101000100110',
 '101101001100',
 '101001110001',
 '111110110000',
 '111101101101',
 '101001000101',
 '101000111100',
 '100011111101',
 '101111011001',
 '100010111101',
 '100011001011',
 '110011011011',
 '110100110111',
 '110101110001',
 '100110100001',
 '100110010110',
 '100010101101',
 '110000111010',
 '100001110011',
 '101111111001',
 '101010001101',
 '100001000110',
 '110000010101',
 '111100100000',
 '111011110011',
 '111101100101',
 '101100011111',
 '100001111111',
 '110011111100',
 '100101001001',
 '110001100010',
 '101001000111',
 '101010000101',
 '101001111011

In [52]:
all_lines[2][4] + 

'1'

In [60]:
a = 1
b = 0
if b:
    print(a, b) 

In [64]:
a  =[1,2,3,14,16,61]
idx = (0,3,5)
res=[]
for i in idx:
    res.append(a[i])
res


[1, 14, 61]